<a href="https://colab.research.google.com/github/TheAgaveFairy/MusicGeneration/blob/main/gpt2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.kaggle.com/code/arnavmishra6996/music-generation-using-decoder-only-model-gpt2

this is the inspiration, more or less copying it but in my own typing + modifications so i understand it

In [3]:
import numpy as np
import pandas as pd
import os

In [4]:
#! pip install pretty_midi
import pretty_midi

In [5]:
#! pip install transformers

In [4]:
"""
!pip install -q kaggle
from google.colab import files
files.upload()

! mkdir ~/.kaggle

! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
#! kaggle datasets list

! kaggle competitions download -c 'name-of-competition'
! mkdir train
! unzip train.zip -d train
"""

"\n!pip install -q kaggle\nfrom google.colab import files\nfiles.upload()\n\n! mkdir ~/.kaggle\n\n! cp kaggle.json ~/.kaggle/\n! chmod 600 ~/.kaggle/kaggle.json\n#! kaggle datasets list\n\n! kaggle competitions download -c 'name-of-competition'\n! mkdir train\n! unzip train.zip -d train\n"

In [7]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("imsparsh/musicnet-dataset")

print("Path to dataset files:", path)
'Path to dataset files: /home/paul-dutton/.cache/kagglehub/datasets/imsparsh/musicnet-dataset/versions/1'

/home/paul-dutton/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|█████████████████████████████████████████████████████████████████████████████████| 21.5G/21.5G [36:26<00:00, 10.6MB/s]

Extracting files...


Path to dataset files: /home/paul-dutton/.cache/kagglehub/datasets/imsparsh/musicnet-dataset/versions/1


In [8]:
"""for dirname, _, filenames in os.walk(path):
    for filename in filenames[:10]: # just make sure this is working
        print(os.path.join(dirname, filename))
"""
def list_files(startpath, trunc=True):
    print(f"root path is:\n{path}\nbeginning walk:")
    for root, dirs, files in os.walk(startpath):
        level = root.replace(startpath, '').count(os.sep)
        indent = ' ' * 4 * (level)
        print('{}{}/'.format(indent, os.path.basename(root)))
        subindent = ' ' * 4 * (level + 1)
        if trunc:
            files = files[:2]
        for f in files:
            print('{}{}'.format(subindent, f))
list_files(path)

root path is:
/home/paul-dutton/.cache/kagglehub/datasets/imsparsh/musicnet-dataset/versions/1
beginning walk:
1/
    musicnet.npz
    musicnet_metadata.csv
    musicnet_midis/
        musicnet_midis/
            Faure/
                2166_gr_f45m1.mid
                2168_gr_f45m3.mid
            Bach/
                2243_vs1_3.mid
                2659_vs2_6.mid
            Dvorak/
                1932_dv96_3.mid
                1919_dvqt10m4.mid
            Cambini/
                2082_quint3f2.mid
                2077_quintBb3.mid
            Haydn/
                2105_op64n5_2.mid
                2104_op64n5_1.mid
            Brahms/
                2151_br25m4.mid
                2148_br25m1.mid
            Mozart/
                1819_k3754a.mid
                1811_kv581b.mid
            Ravel/
                2180_gr_rqtf4.mid
                2179_gr_rqtf3.mid
            Beethoven/
                2510_ps02_02.mid
                2322_ps23_01.mid
            Schubert/
    

In [9]:
temp_file_path = os.path.join(path, 'musicnet/musicnet/test_labels/2382.csv')
#print(temp_file_path)
data = pd.read_csv(temp_file_path)
data.head(3)

,start_time,end_time,instrument,note,start_beat,end_beat,note_value
0,10206,42462,41,61,0.0,1.489583,Dotted Half
1,10206,25054,41,65,0.0,0.489583,Quarter
2,10206,22494,43,46,0.0,0.333333,Dotted Eighth


In [10]:
# let's just pick a file and see what we're dealing with
midi_data = pretty_midi.PrettyMIDI(os.path.join(path,'musicnet_midis/musicnet_midis/Beethoven/2313_qt15_1.mid'))

/home/paul-dutton/miniconda3/lib/python3.12/site-packages/pretty_midi/pretty_midi.py:100: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  warnings.warn(


In [9]:
"""
data = []
for instrument in midi_data.instruments:
    #print(instrument) # Instrument(program=40, is_drum=False, name="Violin1") kinda output
    for note in instrument.notes:
        #print(note) # start, end, pitch, velocity
        velocity = note.velocity
        start_time = note.start
        end_time = note.end
        instrument_name = instrument.name if instrument.name else 'Unnamed'
        note_name = pretty_midi.note_number_to_name(note.pitch)
        start_beat = midi_data.get_beats(start_time)
        end_beat = midi_data.get_beats(end_time)
        note_value = note.pitch

        data.append([start_time, end_time, instrument_name, note_name, start_beat, end_beat, velocity, note_value])

# Create a DataFrame
columns = ['start_time', 'end_time', 'instrument', 'note', 'start_beat', 'end_beat', 'velocity', 'note_value']
df = pd.DataFrame(data, columns=columns)

# Save the DataFrame to a CSV file (optional)
#df.to_csv('midi_data.csv', index=False)

print(df.head())
"""

'\ndata = []\nfor instrument in midi_data.instruments:\n    #print(instrument) # Instrument(program=40, is_drum=False, name="Violin1") kinda output\n    for note in instrument.notes:\n        #print(note) # start, end, pitch, velocity\n        velocity = note.velocity\n        start_time = note.start\n        end_time = note.end\n        instrument_name = instrument.name if instrument.name else \'Unnamed\'\n        note_name = pretty_midi.note_number_to_name(note.pitch)\n        start_beat = midi_data.get_beats(start_time)\n        end_beat = midi_data.get_beats(end_time)\n        note_value = note.pitch\n\n        data.append([start_time, end_time, instrument_name, note_name, start_beat, end_beat, velocity, note_value])\n\n# Create a DataFrame\ncolumns = [\'start_time\', \'end_time\', \'instrument\', \'note\', \'start_beat\', \'end_beat\', \'velocity\', \'note_value\']\ndf = pd.DataFrame(data, columns=columns)\n\n# Save the DataFrame to a CSV file (optional)\n#df.to_csv(\'midi_data.cs

In [11]:
entire_data = []
from tqdm import tqdm
def midi_data_extraction(file_path):
    # Check if the path is a directory
    if os.path.isdir(file_path):
        # Loop through all files in the directory
        for file_name in tqdm(os.listdir(file_path)):
            if file_name.endswith('.mid'):
                # Construct the full file path
                full_path = os.path.join(file_path, file_name)
                # Load the MIDI file
                midi_data = pretty_midi.PrettyMIDI(full_path)

                # Extract data for each note
                for instrument in midi_data.instruments:
                    for note in instrument.notes:
                        start_time = note.start
                        end_time = note.end
                        instrument_name = instrument.name if instrument.name else 'Unnamed'
                        note_name = pretty_midi.note_number_to_name(note.pitch)
                        start_beat = midi_data.get_beats(start_time)
                        end_beat = midi_data.get_beats(end_time)
                        note_value = note.pitch
                        duration = end_time - start_time  # Duration of the note
                        velocity = note.velocity  # Velocity of the note

                        # Add the data to the list
                        entire_data.append([start_time, end_time, instrument_name, note_name, start_beat, end_beat, note_value, duration, velocity])

In [12]:
# path = '/root/.cache/kagglehub/datasets/imsparsh/musicnet-dataset/versions/1'
file_name = os.path.join(path, 'musicnet_midis/musicnet_midis/Beethoven')
# print(file_name)
midi_data_extraction(file_name)
entire_data[:1]

100%|████████████████████████████████████████████████████████████████████████████████████| 157/157 [10:49<00:00,  4.14s/it]


[[np.float64(0.0),
  np.float64(1.2369791666666667),
  'Right',
  'F#4',
  array([  0.       ,   1.25     ,   2.5      ,   3.75     ,   5.       ,
           6.25     ,   7.5      ,   8.75     ,  10.       ,  11.25     ,
          12.5      ,  13.75     ,  15.       ,  16.25     ,  17.5      ,
          18.75     ,  20.       ,  21.25     ,  22.5      ,  23.75     ,
          25.       ,  26.25     ,  27.5      ,  28.75     ,  30.       ,
          31.25     ,  32.5      ,  33.75     ,  35.       ,  36.25     ,
          37.5      ,  38.75     ,  40.       ,  41.25     ,  42.5416665,
          44.0251825,  45.767606 ,  47.017606 ,  48.267606 ,  49.517606 ,
          50.767606 ,  52.017606 ,  53.267606 ,  54.517606 ,  55.767606 ,
          57.017606 ,  58.267606 ,  59.517606 ,  60.767606 ,  62.017606 ,
          63.267606 ,  64.517606 ,  65.767606 ,  67.017606 ,  68.267606 ,
          69.517606 ,  70.671452 ,  71.825298 ,  72.979144 ,  74.13299  ,
          75.286836 ,  76.440682 ,  77.

In [13]:
columns = ['start_time', 'end_time', 'instrument', 'note', 'start_beat', 'end_beat', 'note_value', 'duration', 'velocity']

# Create a DataFrame with the collected data
df = pd.DataFrame(entire_data, columns=columns)

# Save the DataFrame to a CSV file (optional)
df.to_csv('entire_midi_data.csv', index=False)

# Print the first few rows of the DataFrame
print(df.head())

   start_time  end_time instrument note  \
0        0.00  1.236979      Right  F#4   
1        0.00  1.236979      Right   A3   
2        1.25  2.486979      Right  F#4   
3        1.25  2.486979      Right   A3   
4        2.50  3.736979      Right  F#4   

                                          start_beat  \
0  [0.0, 1.25, 2.5, 3.75, 5.0, 6.25, 7.5, 8.75, 1...   
1  [0.0, 1.25, 2.5, 3.75, 5.0, 6.25, 7.5, 8.75, 1...   
2  [1.25, 2.5, 3.75, 5.0, 6.25, 7.5, 8.75, 10.0, ...   
3  [1.25, 2.5, 3.75, 5.0, 6.25, 7.5, 8.75, 10.0, ...   
4  [2.5, 3.75, 5.0, 6.25, 7.5, 8.75, 10.0, 11.25,...   

                                            end_beat  note_value  duration  \
0  [1.2369791666666667, 2.486979166666667, 3.7369...          66  1.236979   
1  [1.2369791666666667, 2.486979166666667, 3.7369...          57  1.236979   
2  [2.486979166666667, 3.736979166666667, 4.98697...          66  1.236979   
3  [2.486979166666667, 3.736979166666667, 4.98697...          57  1.236979   
4  [3.73697916

GPT-2 needs this as text

In [14]:
with open('midi_text_data.txt', 'w') as file:
    file.write('<|startoftext|>\n')
    for index, row in tqdm(df.iterrows()):
        text_line = f"start_time: {row['start_time']}, end_time: {row['end_time']}, instrument: {row['instrument']}, note: {row['note']}, start_beat: {row['start_beat']}, end_beat: {row['end_beat']}, note_value: {row['note_value']}, duration: {row['duration']}, velocity: {row['velocity']}\n"
        file.write(text_line)
    file.write('<|endoftext|>\n')

569506it [12:49, 740.49it/s] 


Here we go

In [17]:
import tensorflow as tf
from transformers import GPT2Tokenizer
from tqdm import tqdm

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token

def load_tf_dataset(file_path, tokenizer, block_size=128, chunk_size=1000000):
    def chunk_generator():
        with open(file_path, 'r', encoding='utf-8') as f:
            while True:
                chunk = f.read(chunk_size)
                if not chunk:
                    break
                yield chunk

    def process_chunk(chunk):
        tokens = tokenizer.tokenize(chunk)
        ids = tokenizer.convert_tokens_to_ids(tokens)
        for i in range(0, len(ids) - block_size + 1, block_size):
            input_ids = ids[i:i+block_size]
            labels = ids[i+1:i+block_size+1] # shifted by 1
            labels = [-100 if token == tokenizer.pad_token_id else token for token in labels] # masking with -100

            yield {
                "input_ids": input_ids,#ids[i:i+block_size],
                "attention_mask": [1] * block_size,
                "labels": labels
            }

    def gen():
        for chunk in tqdm(chunk_generator(), desc="Processing chunks"):
            yield from process_chunk(chunk)

    return tf.data.Dataset.from_generator(
        gen,
        output_signature=(
            {
                "input_ids": tf.TensorSpec(shape=(block_size,), dtype=tf.int32),
                "attention_mask": tf.TensorSpec(shape=(block_size,), dtype=tf.int32),
                "labels": tf.TensorSpec(shape=(block_size,), dtype=tf.int32)
            }
        )
    )

# Load the dataset
train_dataset = load_tf_dataset('midi_text_data.txt', tokenizer)

In [17]:
#!pip install transformers[tf-cpu]

In [18]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
!mkdir "/content/drive/My Drive/MusicGen"

Mounted at /content/drive
mkdir: cannot create directory ‘/content/drive/My Drive/MusicGen’: File exists


In [ ]:
"""
with open('/content/drive/My Drive/MusicGen/midi_text_data.txt', 'w') as f:
  with open('midi_text_data.txt', 'r') as f2:
    f.write(f2.read())
  #f.write('./midi_text_data.txt')
"""

In [19]:
tf.config.list_physical_devices('CPU'),tf.config.list_physical_devices('GPU')

([PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')], [])

In [ ]:
"""
import tensorflow as tf
from transformers import GPT2Tokenizer, TFGPT2LMHeadModel, TFTrainingArguments #TFTrainer,
from transformers import DataCollatorForLanguageModeling
from transformers import TFTrainer# as TFTrainer
from tqdm import tqdm

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

# Define the training arguments
training_args = TFTrainingArguments(
    output_dir="./gpt2-midi",  # Directory where the model and tokenizer will be saved
    overwrite_output_dir=True,
    num_train_epochs=3,  # Number of training epochs
    per_device_train_batch_size=4,  # Batch size for training
    save_steps=10_000,  # Save every 10,000 steps
    save_total_limit=2,  # Keep only the 2 most recent models
    prediction_loss_only=True,  # Only compute the loss during training
    logging_dir='./logs',  # Directory for logs
    logging_steps=500  # Log every 500 steps
)

#Gemini
strategy = tf.distribute.MirroredStrategy(['CPU:0'])

# Create the model within the strategy scope
with strategy.scope():
    model = TFGPT2LMHeadModel.from_pretrained('gpt2')
    #/Gemini

# Initialize the Trainer
trainer = TFTrainer(
    model=model,#TFGPT2LMHeadModel.from_pretrained('gpt2'),
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset
)
"""
'This code was deprecated on or about 2021.'

In [ ]:
"""
# Start training
trainer.train()

# Save the model and tokenizer
trainer.save_model('./gpt2-midi')
tokenizer.save_pretrained('./gpt2-midi')
"""

In [21]:
#Claude 3.5Sonnet to the rescue, maybe
import tensorflow as tf
from transformers import GPT2Tokenizer, TFGPT2LMHeadModel
from transformers import DataCollatorForLanguageModeling, DefaultDataCollator
from tqdm import tqdm
import os

# Initialize tokenizer and data collator
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
"""
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=True # changed from False
)
"""
data_collator = DefaultDataCollator()

# Training configuration
class TrainingConfig:
    output_dir = "./gpt2-midi"
    num_train_epochs = 3
    per_device_train_batch_size = 4
    save_steps = 10_000
    save_total_limit = 2
    logging_dir = './logs'
    logging_steps = 500

config = TrainingConfig()

# Create output directory if it doesn't exist
os.makedirs(config.output_dir, exist_ok=True)
os.makedirs(config.logging_dir, exist_ok=True)

# Set up distributed training strategy
strategy = tf.distribute.MirroredStrategy()
print(f"Number of devices: {strategy.num_replicas_in_sync}")

# Create the model within the strategy scope
with strategy.scope():
    model = TFGPT2LMHeadModel.from_pretrained('gpt2')

    # Set up the optimizer
    optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)

    # Compile the model
    model.compile(
        optimizer=optimizer,
        loss=model.compute_loss
    )

# Prepare the dataset
def prepare_dataset_for_training(dataset, batch_size):
    """Convert dataset to tf.data.Dataset format"""
    dataset = dataset.shuffle(1000)
    dataset = dataset.batch(batch_size, drop_remainder=True)
    """
    # Modify attention mask shape
    dataset = dataset.map(lambda x: {
        **x,
        "attention_mask": tf.reshape(x["attention_mask"], (tf.shape(x["attention_mask"])[0], 1, 1, tf.shape(x["attention_mask"])[2]))  # Reshape to (batch_size, 1, 1, seq_len)
    })
    """
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    return dataset

train_dataset = prepare_dataset_for_training(
    train_dataset,  # Your original train_dataset
    config.per_device_train_batch_size
)

# Custom callback for model saving
class ModelCheckpoint(tf.keras.callbacks.Callback):
    def __init__(self, save_steps, output_dir, save_total_limit):
        super().__init__()
        self.save_steps = save_steps
        self.output_dir = output_dir
        self.save_total_limit = save_total_limit
        self.step = 0

    def on_batch_end(self, batch, logs=None):
        self.step += 1
        if self.step % self.save_steps == 0:
            # Save the model
            checkpoint_dir = os.path.join(
                self.output_dir,
                f"checkpoint-{self.step}"
            )
            self.model.save_pretrained(checkpoint_dir)

            # Remove old checkpoints if exceeding limit
            checkpoints = sorted([
                d for d in os.listdir(self.output_dir)
                if d.startswith("checkpoint-")
            ])
            while len(checkpoints) > self.save_total_limit:
                checkpoint_to_remove = os.path.join(
                    self.output_dir,
                    checkpoints.pop(0)
                )
                tf.io.gfile.rmtree(checkpoint_to_remove)

# Set up callbacks
callbacks = [
    ModelCheckpoint(
        save_steps=config.save_steps,
        output_dir=config.output_dir,
        save_total_limit=config.save_total_limit
    ),
    tf.keras.callbacks.TensorBoard(
        log_dir=config.logging_dir,
        update_freq=config.logging_steps
    )
]

# Training
print("Starting training...")
history = model.fit(
    train_dataset,
    epochs=config.num_train_epochs,
    callbacks=callbacks
)

# Save the final model and tokenizer
model.save_pretrained(config.output_dir)
tokenizer.save_pretrained(config.output_dir)
print(f"Model saved to {config.output_dir}")

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
Number of devices: 1


All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


Starting training...
Epoch 1/3


2025-02-10 19:14:39.445264: W tensorflow/core/framework/dataset.cc:993] Input of GeneratorDatasetOp::Dataset will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.


INFO:tensorflow:Error reported to Coordinator: in user code:

    File "/home/paul-dutton/miniconda3/lib/python3.12/site-packages/transformers/modeling_tf_utils.py", line 1588, in compute_loss  *
        return super().compute_loss(*args, **kwargs)
    File "/home/paul-dutton/miniconda3/lib/python3.12/site-packages/tf_keras/src/engine/training.py", line 1206, in compute_loss  **
        return self.compiled_loss(
    File "/home/paul-dutton/miniconda3/lib/python3.12/site-packages/tf_keras/src/engine/compile_utils.py", line 275, in __call__
        y_t, y_p, sw = match_dtype_and_rank(y_t, y_p, sw)
    File "/home/paul-dutton/miniconda3/lib/python3.12/site-packages/tf_keras/src/engine/compile_utils.py", line 854, in match_dtype_and_rank
        if (y_t.dtype.is_floating and y_p.dtype.is_floating) or (

    AttributeError: 'NoneType' object has no attribute 'dtype'
Traceback (most recent call last):
  File "/home/paul-dutton/miniconda3/lib/python3.12/site-packages/tensorflow/python/traini

AttributeError: in user code:

    File "/home/paul-dutton/miniconda3/lib/python3.12/site-packages/tf_keras/src/engine/training.py", line 1398, in train_function  *
        return step_function(self, iterator)
    File "/home/paul-dutton/miniconda3/lib/python3.12/site-packages/transformers/modeling_tf_utils.py", line 1588, in compute_loss  *
        return super().compute_loss(*args, **kwargs)
    File "/home/paul-dutton/miniconda3/lib/python3.12/site-packages/tf_keras/src/engine/training.py", line 1206, in compute_loss  **
        return self.compiled_loss(
    File "/home/paul-dutton/miniconda3/lib/python3.12/site-packages/tf_keras/src/engine/compile_utils.py", line 275, in __call__
        y_t, y_p, sw = match_dtype_and_rank(y_t, y_p, sw)
    File "/home/paul-dutton/miniconda3/lib/python3.12/site-packages/tf_keras/src/engine/compile_utils.py", line 854, in match_dtype_and_rank
        if (y_t.dtype.is_floating and y_p.dtype.is_floating) or (

    AttributeError: 'NoneType' object ha